In [20]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import scipy

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [21]:
import layers as custom_layers

In [2]:
# File Paths
imgIdxCsvPath = './MRNet/MRNet-v1.0/similar.csv'
MRI_Path = './MRNet/MRNet-v1.0/train/axial/{}.npy'

In [3]:
# Get indices of qualified training images in MRNet.
imageIndices = []
with open(imgIdxCsvPath) as file:
    fileReader = csv.reader(file)
    # Find all the rows with abnormal as 0 and store their indices
    for row in fileReader:
        index = str(row[0])
        while(len(index) < 4):
            index = '0' + index
        imageIndices.append(index)

In [4]:
print(imageIndices)

['0000', '0002', '0009', '0013', '0015', '0020', '0033', '0070', '0086', '0143', '0170', '0171', '0174', '0179', '0184', '0204', '0209', '0215', '0257', '0262', '0274', '0275', '0286', '0302', '0317', '0321', '0351', '0362', '0366', '0367', '0372', '0377', '0396', '0431', '0440', '0445', '0448', '0460', '0472', '0474', '0479', '0491', '0495', '0496', '0500', '0525', '0531', '0532', '0573', '0575', '0580', '0599', '0605', '0607', '0609', '0644', '0648', '0662', '0666', '0669', '0674', '0701', '0702', '0716', '0724', '0739', '0751', '0769', '0805', '0806', '0815', '0818', '0827', '0867', '0876', '0880', '0885', '0903', '0941', '0957', '0969', '0981', '1020', '1026', '1039', '1060', '1079', '1106', '1116', '1121']


In [5]:
# Check which image has most slices, then we use it as our atlas/target/fixed image.
maxSlice = 0
targetIndex = None
for index in imageIndices:
    img = np.load(MRI_Path.format(index))
    if (img.shape[0] > maxSlice):
        targetIndex = index
        maxSlice = img.shape[0]
    
print(maxSlice)
print(targetIndex)

52
0701


In [6]:
# Load fixed image.
fixedImg = np.load(MRI_Path.format(targetIndex))

In [7]:
# Load moving images and zero-pad them.
movingImgs = []
for index in imageIndices:
    if index == targetIndex:
        continue
    img = np.load(MRI_Path.format(index))
    numSlicesToPaddle = maxSlice - img.shape[0]
    img = np.pad(img, ((0, numSlicesToPaddle), (0, 0), (0, 0)))
    movingImgs.append(img)

In [18]:
print(fixedImg.shape)
print(type(fixedImg.shape))

(52, 256, 256)
<class 'tuple'>


In [23]:
# Construct NN
nFeats = 1
for i in range(len(fixedImg.shape)):
    nFeats *= fixedImg.shape[i]
print(nFeats)
inputs = keras.Input(shape = (nFeats,))
mid_layer_1 = layers.Dense(4096, activation="relu")(inputs)
mid_layer_2 = layers.Dense(256, activation="relu")(mid_layer_1)
outputs = layers.Dense(12, activation="relu")(mid_layer_2)


3407872


ResourceExhaustedError: OOM when allocating tensor with shape[3407872,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [11]:
n = np.array([1., 2., 4.])
t = tf.constant(n)
print(t.shape)
print(t.shape.as_list())

(3,)
[3]
